In [1]:
!pip install -q 'git+https://github.com/drscook/voting_predictor'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 5.2 MB/s 
     |████████████████████████████████| 16.6 MB 36.4 MB/s 
     |████████████████████████████████| 7.8 MB 56.1 MB/s 
     |████████████████████████████████| 1.6 MB 71.9 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 132 kB 55.3 MB/s 


In [2]:
%reload_ext google.colab.data_table
%reload_ext autotime
%reload_ext autoreload
%autoreload 2

root_path = '/content/drive/'
local = False
local = True
if local:
    import sys, google.colab
    google.colab.drive.mount(root_path)
    sys.path.insert(0, root_path + 'MyDrive/helpers')
    sys.path.insert(0, root_path + 'MyDrive/voting_predictor')
    root_path += 'MyDrive/gerrymandering/2022-12/'
# else:
#     !pip install -q --force-reinstall --no-deps 'git+https://github.com/drscook/helpers' 'git+https://github.com/drscook/voting_predictor'

from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp

self = vp.Redistricter(
    census_api_key = 'dccb7bb4b7df5dff59d2d99c859016f973197e4e',
    bg_project_id = 'redistricting-361203',
    state = 'RI',
    root_path = root_path,
)

self.get_crosswalks()#overwrite=True)
self.get_assignments()
self.get_shapes()
# self.combine()
# vp.test.f(5)
# ut.

Mounted at /content/drive/


/content/drive/MyDrive/helpers/helpers/utilities.py:14: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if X is None or X is np.nan or X is '':
/content/drive/MyDrive/helpers/helpers/utilities.py:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if X is None or X is np.nan or X is '':


Elapsed time: 0.3639 seconds
Elapsed time: 0.1534 seconds
getting shapes.RI2020


,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry
0,44,003,020601,1023,440030206011023,Block 1023,G5040,None,None,None,S,72653,0,+41.7235508,-071.5427810,23,53,"POLYGON ((-71.54609 41.72285, -71.54609 41.722..."
1,44,003,020903,3011,440030209033011,Block 3011,G5040,None,None,None,S,277227,0,+41.6370196,-071.5366212,29,96,"POLYGON ((-71.53919 41.63524, -71.53862 41.638..."
2,44,003,021402,1021,440030214021021,Block 1021,G5040,None,None,None,S,9580,0,+41.7287911,-071.4050826,6,21,"POLYGON ((-71.40607 41.72902, -71.40511 41.729..."


Elapsed time: 10.2627 seconds


KeyError: ignored

In [9]:
from helpers.common_imports import *
from helpers import utilities as ut
import geopandas as gpd
zip_file = '/content/tl_2010_01013_tabblock10.zip'
X = gpd.read_file(zip_file)
ut.prep(X).columns
isinstance(X, pd.DataFrame)
t = type(X)
t in [pd.DataFrame, gpd.GeoDataFrame]

True

In [37]:
pd.DataFrame(self.census.acs5.get(
    'NAME',
    geo={'for': 'tract:*', 'in': 'state:11'},
    year=2018
))

,NAME,state,county,tract
0,"Census Tract 97, District of Columbia, Distric...",11,001,009700
1,"Census Tract 98.07, District of Columbia, Dist...",11,001,009807
2,"Census Tract 5.02, District of Columbia, Distr...",11,001,000502
3,"Census Tract 18.03, District of Columbia, Dist...",11,001,001803
4,"Census Tract 95.08, District of Columbia, Dist...",11,001,009508
...,...,...,...,...
174,"Census Tract 55, District of Columbia, Distric...",11,001,005500
175,"Census Tract 62.02, District of Columbia, Dist...",11,001,006202
176,"Census Tract 78.04, District of Columbia, Dist...",11,001,007804
177,"Census Tract 80.01, District of Columbia, Dist...",11,001,008001


In [ ]:
# df.geometry = df.geometry.buffer(0).apply(orient, args=(1,))
self.bq.df_to_tbl(df, tbl)

'shapes.RI'

In [ ]:
@codetiming.Timer()
def f():
    return 2

f()

Elapsed time: 0.0001 seconds


2

In [ ]:
levels = {
    'state':2,
    'county':3,
    'tract':6,
    'block_group':1,
    'blk':4,
}
X = df.copy()
w = [ut.rjust(df[f'{l}_2010'], d) for l, d in levels.items() if l!='block_group']
display(w[0]+w[1]+w[2]+w[3])
# type(w[0])
# w = df['blk_2010']
# w
# isinstance(w, pd.Series)
# type(w)

index
0          050910209001225
1          050910210007000
2          050910210007031
3          050910210007031
4          050910210008022
                ...       
1036388    485079503025022
1036389    485079503025023
1036390    485079503025024
1036391    485079503025025
1036392    485079503025026
Length: 1036393, dtype: object

time: 5.34 s (started: 2022-12-24 08:31:37 +00:00)


In [ ]:
!pip install --force-reinstall 'git+https://github.com/drscook/REU_temp'
from temp import ut

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/drscook/REU_temp to /tmp/pip-install-hq5yk4dk/reu-temp_63a2d4ef244e4081b4596b78132f1c58
  Running command git clone -q https://github.com/drscook/REU_temp /tmp/pip-install-hq5yk4dk/reu-temp_63a2d4ef244e4081b4596b78132f1c58
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 43 kB 2.3 MB/s 
  Created wheel for REU-temp: filename=reu_temp-0.1.0-py2.py3-none-any.whl size=1147 sha256=c27bd9700e60edac66fd2cbfb5e7193d9d8941b5c58154478497360c4cd0dd80
  Stored in directory: /tmp/pip-ephem-wheel-cache-4c_c20og/wheels/e8/c2/b9/7e6d3335fac2bc78dd184be935c3dd2759267588fa21e58746
  Created wheel for shutils: filename=shutils-0.1.0-py3-none-any.whl size=3516 sha256=e48f5d154b17004e84010db0205dc9443140c4e6e0fcb88085820339640a6817
  Stored in directory: /root/.cac

In [ ]:
from REU_temp import ut
ut.f(45)

ModuleNotFoundError: ignored

In [ ]:
!pip install --force-reinstall 'git+https://github.com/drscook/test_drscook'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/drscook/test_drscook to /tmp/pip-req-build-_4ki3aq6
  Running command git clone -q https://github.com/drscook/test_drscook /tmp/pip-req-build-_4ki3aq6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/drscook/test_drscook2 to /tmp/pip-install-zkbpzjqp/test-drscook2_f132ce5346b34e23ad939165b6c69d55
  Running command git clone -q https://github.com/drscook/test_drscook2 /tmp/pip-install-zkbpzjqp/test-drscook2_f132ce5346b34e23ad939165b6c69d55
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 43 kB 2.1 MB/s 
  Created wheel for test-drscook: filename=test_drscook-1.0.0-py2.py3-none-any.whl size=2369 sha256=1af4c507f7347e1ef76c52c1b9db32f0d3db26205

In [ ]:
# !pip install ipython-autotime
!pip install jupyter-autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 18.0 MB/s 


In [ ]:
%load_ext autotime
import time
for i in range(10):
    print(i)
    time.sleep(.1)

0
1
2
3
4
5
6
7
8
9


In [1]:
from helpers.common_imports import *
from helpers import utilities as ut
import geopandas as gpd
zip_file = '/content/tl_2010_01013_tabblock10.zip'
X = gpd.read_file(zip_file)
ut.prep(X).columns

ValueError: ignored

ValueError: ignored